In [6]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import pandas as pd
from tqdm import tqdm
import os
import os
from sklearn.metrics import confusion_matrix

In [7]:
brand = ["1","2","3"]
brand_num = 0

train_label_obj = np.load(f"./AUROCscore/labels/brand3/train_labels.npy",allow_pickle=True).item()
test_label_obj = np.load(f"./AUROCscore/labels/brand3/labels.npy",allow_pickle=True).item()
train_scores = np.load(f"./AUROCscore/scores/brand3/tranad_3_epochs5_batch32/train_scores.npy",allow_pickle=True).item()
test_scores = np.load(f"./AUROCscore/scores/brand3/tranad_3_epochs5_batch32/test_scores.npy",allow_pickle=True).item()

In [22]:
train_scores.update(test_scores) # train, test 하나로 합침.
ind_car_num_list = list(train_label_obj.keys())
ood_car_num_list = []
print(ind_car_num_list)

for k,v in test_label_obj.items():
    if test_label_obj[k]==1:
        ood_car_num_list.append(k)
    else :
        ind_car_num_list.append(k)

all_car_num_list = set(ind_car_num_list + ood_car_num_list)
print(len(all_car_num_list))

[201, 202, 207, 208, 211, 214, 216, 218, 220, 221, 222, 223, 224, 227, 228, 229, 231, 232, 233, 234, 238, 242, 244, 245, 246, 248, 250]
49


In [9]:
rows = []

for car_id, snippet_scores in train_scores.items():
    if car_id in ind_car_num_list:
        label = 0
    elif car_id in ood_car_num_list:
        label = 1
    else :
        continue

    for s in snippet_scores:
        rows.append([car_id,label,float(s)])

all_snippet_df = pd.DataFrame(rows,columns=['car','label','rec_error'])

In [ ]:
AUC_fivefold_list = []
# for i in range(5):
for i in range(5):
    fold_num = i
    
    validation_car_number = ind_car_num_list[:int(fold_num * len(ind_car_num_list) / 5)] + ind_car_num_list[int((fold_num + 1) * len(ind_car_num_list) / 5):] + ood_car_num_list[int(fold_num * len(ood_car_num_list) / 5):int((fold_num + 1) * len(ood_car_num_list) / 5)]
    # test_car_number = ind_car_num_list[int(fold_num * len(ind_car_num_list) / 5):int((fold_num + 1) * len(ind_car_num_list) / 5)] + ood_car_num_list[:int(fold_num * len(ood_car_num_list) / 5)] + ood_car_num_list[int((fold_num + 1) * len(ood_car_num_list) / 5):]
    test_car_number = ind_car_num_list[int(fold_num * len(ind_car_num_list) / 5):int((fold_num + 1) * len(ind_car_num_list) / 5)] + ood_car_num_list[:int(fold_num * len(ood_car_num_list) / 5)] + ood_car_num_list[int((fold_num + 1) * len(ood_car_num_list) / 5):]
    

    
    train_result = all_snippet_df[all_snippet_df['car'].isin(validation_car_number)].copy()
    test_result = all_snippet_df[all_snippet_df['car'].isin(test_car_number)].copy()      
    
    train_res_csv = train_result[['label','car','rec_error']].to_numpy()
    test_res_csv = test_result[['label','car','rec_error']].to_numpy()

    train_df = pd.DataFrame(train_res_csv, columns=['label', 'car', 'rec_error'])
    test_df  = pd.DataFrame(test_res_csv,  columns=['label', 'car', 'rec_error'])
    
    # car, label 타입 정리
    train_df['car'] = train_df['car'].astype(int)
    test_df['car']  = test_df['car'].astype(int)
    train_df['rec_error'] = train_df['rec_error'].astype(float)
    test_df['rec_error']  = test_df['rec_error'].astype(float)

    data_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

    ind_car_score = []
    ood_car_score = []
    for each_car_num in test_car_number:
        this_car_score = data_df.loc[
            data_df['car'] == each_car_num, 'rec_error'
        ].mean()
            
        if each_car_num in ind_car_num_list:
            ind_car_score.append(this_car_score)
        else:
            ood_car_score.append(this_car_score)

    
    label = [1] * len(ood_car_score) + [0] * len(ind_car_score)
    score = np.hstack((ood_car_score, ind_car_score))

    print(label)
    print(score)
    fpr, tpr, thresholds = metrics.roc_curve(label, score, pos_label=1)

    plt.plot(fpr, tpr)
    plt.plot([0,1],[0,1],"k--")

    plt.show()
    AUC = auc(fpr, tpr)
    print(AUC)
    AUC_fivefold_list.append(AUC)
    
print('AUC mean ', np.mean(AUC_fivefold_list))


Empty DataFrame
Columns: [label, car, rec_error]
Index: []
Empty DataFrame
Columns: [label, car, rec_error]
Index: []
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]


ValueError: Input contains NaN.

In [11]:
import numpy as np

AUC_mean = np.mean(AUC_fivefold_list)
AUC_std = np.std(AUC_fivefold_list)

print("AUC mean:", AUC_mean)
print("AUC std:", AUC_std)


AUC mean: nan
AUC std: nan


/home/junki/miniconda3/envs/tranad/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/junki/miniconda3/envs/tranad/lib/python3.10/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/junki/miniconda3/envs/tranad/lib/python3.10/site-packages/numpy/_core/_methods.py:223: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/junki/miniconda3/envs/tranad/lib/python3.10/site-packages/numpy/_core/_methods.py:181: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/junki/miniconda3/envs/tranad/lib/python3.10/site-packages/numpy/_core/_methods.py:215: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

mean_fpr = np.linspace(0, 1, 100)
tprs = []
AUC_fivefold_list = []

epoch = 5    # 필요에 따라 수정
lr = 0.001   # 필요에 따라 수정

for i in range(5):
    fold_num = i
    
    # ---------------------------
    # 1) validation / test car split (네가 쓰던 방식 그대로)
    # ---------------------------
    validation_car_number = (
        ind_car_num_list[:int(fold_num * len(ind_car_num_list) / 5)]
        + ind_car_num_list[int((fold_num + 1) * len(ind_car_num_list) / 5):]
        + ood_car_num_list[
            int(fold_num * len(ood_car_num_list) / 5)
            : int((fold_num + 1) * len(ood_car_num_list) / 5)
        ]
    )

    test_car_number = (
        ind_car_num_list[
            int(fold_num * len(ind_car_num_list) / 5)
            : int((fold_num + 1) * len(ind_car_num_list) / 5)
        ]
        + ood_car_num_list[:int(fold_num * len(ood_car_num_list) / 5)]
        + ood_car_num_list[int((fold_num + 1) * len(ood_car_num_list) / 5):]
    )
    
    # ---------------------------
    # 2) 이 fold에서의 train / test snippet 결과 분리
    #    all_snippet_df는 [label, car, rec_error, ...] 컬럼을 가진 DF라고 가정
    # ---------------------------
    train_result = all_snippet_df[all_snippet_df['car'].isin(validation_car_number)].copy()
    test_result  = all_snippet_df[all_snippet_df['car'].isin(test_car_number)].copy()      
    
    train_res_csv = train_result[['label','car','rec_error']].to_numpy()
    test_res_csv  = test_result[['label','car','rec_error']].to_numpy()

    train_df = pd.DataFrame(train_res_csv, columns=['label', 'car', 'rec_error'])
    test_df  = pd.DataFrame(test_res_csv,  columns=['label', 'car', 'rec_error'])

    # 타입 정리
    train_df['car'] = train_df['car'].astype(int)
    test_df['car']  = test_df['car'].astype(int)
    train_df['rec_error'] = train_df['rec_error'].astype(float)
    test_df['rec_error']  = test_df['rec_error'].astype(float)

    # train + test 전체 snippet 레벨 데이터
    data_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

    # ---------------------------
    # 3) car 단위 score 계산 (각 차량의 rec_error 평균)
    # ---------------------------
    ind_car_score = []
    ood_car_score = []

    for each_car_num in test_car_number:
        this_car_score = data_df.loc[
            data_df['car'] == each_car_num, 'rec_error'
        ].mean()
            
        if each_car_num in ind_car_num_list:
            ind_car_score.append(this_car_score)
        else:
            ood_car_score.append(this_car_score)

    # label: OOD=1, IND=0
    label = [1] * len(ood_car_score) + [0] * len(ind_car_score)
    score = np.hstack((ood_car_score, ind_car_score))

    # ---------------------------
    # 4) ROC / AUC 계산 + fold별 ROC 저장
    # ---------------------------
    fpr, tpr, thresholds = metrics.roc_curve(label, score, pos_label=1)
    auc_fold = metrics.auc(fpr, tpr)
    AUC_fivefold_list.append(auc_fold)

    # mean_fpr grid에 맞춰 보간 (mean ROC용)
    tpr_interp = np.interp(mean_fpr, fpr, tpr)
    tpr_interp[0] = 0.0
    tprs.append(tpr_interp)

    # 개별 fold ROC도 같이 그림
    plt.plot(fpr, tpr, alpha=0.3, label=f"Fold {i} AUC = {auc_fold:.3f}")

# ---------------------------
# 5) Fold 평균 ROC + 표준편차 밴드
# ---------------------------
mean_tpr = np.mean(tprs, axis=0)
std_tpr = np.std(tprs, axis=0)
mean_tpr[-1] = 1.0  # 끝점 (1,1) 보정

plt.plot(
    mean_fpr,
    mean_tpr,
    color="blue",
    lw=2,
    label=f"Mean ROC (AUC={np.mean(AUC_fivefold_list):.3f})"
)
plt.fill_between(
    mean_fpr,
    mean_tpr - std_tpr,
    mean_tpr + std_tpr,
    color="blue",
    alpha=0.2,
    label="± 1 std"
)

plt.plot([0,1], [0,1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC with Mean ± Std (5-Fold), EPOCH={epoch}, lr={lr}")
plt.legend()
plt.show()

print("Fold AUCs:", AUC_fivefold_list)
print("AUC mean ", np.mean(AUC_fivefold_list))


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]


ValueError: Input contains NaN.